<a href="https://colab.research.google.com/github/keto08/covid-19/blob/master/Household_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pydot
from networkx.drawing.nx_pydot import graphviz_layout


In [ ]:
class Neighborhood:
  def __init__(self, NH, dH, fX):
    self.NH = NH
    self.dH = dH
    self.fX = fX

    # CONTACT NETWORK
    U = nx.Graph()

    # HOUSEHOLDS
    H = {}
    for i in range(self.NH):
      n = rg.poisson(self.dH-1) + 1
      G = nx.complete_graph(n)
      for e in G.edges:
        G.edges[e]['kind'] = 'HOME'      
      H[i] = n
      U = nx.union(U, G, rename=(None, f'{i}.'))
    self.H = pd.Series(H)
    self.U = U
    self.connect()

  def connect(self):
    households = list(self.H.index)
    
    connections = np.round(self.fX*self.NH).astype(int)
    for _ in range(connections):
      chosen = rg.choice(a=households, size=2, replace=False)
      m0 = rg.choice(self.H[chosen[0]])
      m1 = rg.choice(self.H[chosen[1]])
      self.U.add_edge(f'{chosen[0]}.{m0}',f'{chosen[1]}.{m1}', kind='AWAY')
    self.C = [g for g in sorted(nx.connected_components(self.U), key=len, reverse=True)]

  

In [ ]:
def infect(IG, T, seed, day):
  #print('SEED:', seed)
  nbs = [n for n in nb.U.neighbors(seed)]
  #print('NEIGHBORS:', nbs)
  infections = [n for n in nbs if rg.random() < T[nb.U.edges[(seed,n)]['kind']] and n not in IG ]
  #print(f'INFECTIONS [DAY {day}]:', infections)
  for infection in infections:
    IG.add_node(infection, day=day)
    IG.add_edge(seed, infection, day=day)
  #print(f'DAY {day}', IG.edges)
  return IG

In [ ]:
def get_df(IG, seed, day):
  data = []
  for k in IG.out_degree:
    p = k[0].split('.')
    d = {
        'seed': seed,
        'day': day,
        'household': p[0],
        'member': p[1],
        'successors': k[1]
    }
    data.append(d)
  return pd.DataFrame(data)

In [ ]:
def simulate_infection(nb, T, DAYS=60, RUNS=256):
  data = pd.DataFrame()

  for run in range(RUNS):
    IG = nx.DiGraph()
    seed = rg.choice(nb.U.nodes)
    IG.add_node(seed, day=0)
    print(f'\tSEED[{run}/{RUNS}]:', seed)

    for day in range(DAYS):
      infected = [node for node in IG.nodes]
      for i in infected:
        IG = infect(IG, T, i, day=day)
      df = get_df(IG, seed, day)
      data = data.append(df, ignore_index=True)
    
  return data

In [ ]:
# PARAMETERS
NH = 512 # NUMBER OF HOUSEHOLDS
dH = 4 # MEAN HOUSEHOLD SIZE
T = {
  'HOME': 0.5,
  'AWAY': 0.5
}

CONFIGS = 32
RUNS = 128

In [ ]:
fX = 1 # MEAN X-HOUSEHOLD CONNECTIONS

rg = np.random.default_rng()
data_df = pd.DataFrame()
for _ in range(CONFIGS):
  print(f"CONFIG[{_}/{CONFIGS}]")
  nb = Neighborhood(NH, dH, fX)
  data = simulate_infection(nb, T, DAYS=30, RUNS=RUNS)
  data_df = data_df.append(data, ignore_index=True)

data_df.to_csv(f'df_f{fX}.csv')

CONFIG[0/32]
	SEED[0/128]: 481.0
	SEED[1/128]: 248.1
	SEED[2/128]: 323.3
	SEED[3/128]: 302.4
	SEED[4/128]: 352.1
	SEED[5/128]: 158.3
	SEED[6/128]: 17.2
	SEED[7/128]: 196.6
	SEED[8/128]: 52.0
	SEED[9/128]: 184.7
	SEED[10/128]: 58.0
	SEED[11/128]: 420.2
	SEED[12/128]: 307.4
	SEED[13/128]: 404.0
	SEED[14/128]: 25.2
	SEED[15/128]: 286.3
	SEED[16/128]: 377.0
	SEED[17/128]: 331.0
	SEED[18/128]: 48.5
	SEED[19/128]: 120.1
	SEED[20/128]: 14.4
	SEED[21/128]: 214.0
	SEED[22/128]: 16.2
	SEED[23/128]: 162.2
	SEED[24/128]: 60.4
	SEED[25/128]: 378.2
	SEED[26/128]: 263.0
	SEED[27/128]: 312.1
	SEED[28/128]: 448.1
	SEED[29/128]: 243.1
	SEED[30/128]: 339.1
	SEED[31/128]: 459.1
	SEED[32/128]: 97.3
	SEED[33/128]: 480.2
	SEED[34/128]: 212.1
	SEED[35/128]: 2.0
	SEED[36/128]: 425.3
	SEED[37/128]: 382.5
	SEED[38/128]: 376.3
	SEED[39/128]: 137.1
	SEED[40/128]: 392.3
	SEED[41/128]: 183.1
	SEED[42/128]: 388.4
	SEED[43/128]: 155.3
	SEED[44/128]: 386.0
	SEED[45/128]: 491.1
	SEED[46/128]: 133.0
	SEED[47/128]: 302.0
